# Build a Multi-Agent Solution

In this notebook, you'll create a project that orchestrates multiple AI agents using Azure AI Foundry Agent Service. You'll design an AI solution that assists with ticket triage. The connected agents will assess the ticket's priority, suggest a team assignment, and determine the level of effort required to complete the ticket.

This implementation is based on the [Microsoft Learn: Build a Multi-Agent Solution](https://microsoftlearning.github.io/mslearn-ai-agents/Instructions/03b-build-multi-agent-solution.html) tutorial.

## Setup and Configuration

First, let's import the required libraries and set up our connection to Azure AI Foundry.

In [1]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder, ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential

# Load environment variables
load_dotenv(override=True)

# Get project configuration
project_endpoint = os.environ["AIPROJECT_ENDPOINT"]
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model Deployment: {model_deployment}")

Project Endpoint: https://short-hack-ai-foundry.services.ai.azure.com/api/projects/firstProject
Model Deployment: gpt-4o


## Connect to Azure AI Foundry Project

Create a client connection to your Azure AI Foundry project.

In [2]:
try:
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=DefaultAzureCredential(),
    )
    print("✓ Successfully connected to Azure AI Foundry project")
except Exception as e:
    print(f"× Error connecting to project: {str(e)}")

✓ Successfully connected to Azure AI Foundry project


## Create Specialized Agents

We'll create three specialized agents, each with a specific role in the ticket triage process:

1. **Priority Agent**: Assesses the urgency of support tickets
2. **Team Agent**: Determines which team should handle the ticket
3. **Effort Agent**: Estimates the work required to complete the ticket

### Create the Priority Agent

This agent will assess how urgent a ticket is based on its description.

In [3]:
# Create an agent to prioritize support tickets
priority_agent_name = "priority_agent"
priority_agent_instructions = """
Assess how urgent a ticket is based on its description.

Respond with one of the following levels:
- High: User-facing or blocking issues
- Medium: Time-sensitive but not breaking anything
- Low: Cosmetic or non-urgent tasks

Only output the urgency level and a very brief explanation.
"""

priority_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=priority_agent_name,
    instructions=priority_agent_instructions
)

print(f"Created priority agent, ID: {priority_agent.id}")

Created priority agent, ID: asst_Wihug0zX4JJdThzOr7JesLl0


### Create the Team Agent

This agent will determine which team should handle the ticket.

In [4]:
# Create an agent to assign tickets to the appropriate team
team_agent_name = "team_agent"
team_agent_instructions = """
Decide which team should own each ticket.

Choose from the following teams:
- Frontend
- Backend
- Infrastructure
- Marketing

Base your answer on the content of the ticket. Respond with the team name and a very brief explanation.
"""

team_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=team_agent_name,
    instructions=team_agent_instructions
)

print(f"Created team agent, ID: {team_agent.id}")

Created team agent, ID: asst_7lZIJksbVyUmVARFQWFWLYex


### Create the Effort Agent

This agent will estimate how much work is required to complete the ticket.

In [5]:
# Create an agent to estimate effort for a support ticket
effort_agent_name = "effort_agent"
effort_agent_instructions = """
Estimate how much work each ticket will require.

Use the following scale:
- Small: Can be completed in a day
- Medium: 2-3 days of work
- Large: Multi-day or cross-team effort

Base your estimate on the complexity implied by the ticket. Respond with the effort level and a brief justification.
"""

effort_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=effort_agent_name,
    instructions=effort_agent_instructions
)

print(f"Created effort agent, ID: {effort_agent.id}")

Created effort agent, ID: asst_JHwigBWbFleclVUExSwPGi1N


## Create Connected Agent Tools

Now we'll create ConnectedAgentTool objects for each specialized agent. These tools allow the agents to be used by other agents in a coordinated workflow.

In [6]:
# Create connected agent tools for the support agents
priority_agent_tool = ConnectedAgentTool(
    id=priority_agent.id, 
    name=priority_agent_name, 
    description="Assess the priority of a ticket"
)
    
team_agent_tool = ConnectedAgentTool(
    id=team_agent.id, 
    name=team_agent_name, 
    description="Determines which team should take the ticket"
)
    
effort_agent_tool = ConnectedAgentTool(
    id=effort_agent.id, 
    name=effort_agent_name, 
    description="Determines the effort required to complete the ticket"
)

print("✓ Created connected agent tools for all specialized agents")

✓ Created connected agent tools for all specialized agents


## Create the Triage Coordinator Agent

Now we'll create a primary agent that coordinates the ticket triage process. This agent will use the connected agents to analyze support tickets comprehensively.

In [7]:
# Create an agent to triage support ticket processing by using connected agents
triage_agent_name = "triage-agent"
triage_agent_instructions = """
Triage the given ticket. Use the connected tools to determine the ticket's priority, 
which team it should be assigned to, and how much effort it may take.
"""

triage_agent = project_client.agents.create_agent(
    model=model_deployment,
    name=triage_agent_name,
    instructions=triage_agent_instructions,
    tools=[
        priority_agent_tool.definitions[0],
        team_agent_tool.definitions[0],
        effort_agent_tool.definitions[0]
    ]
)

print(f"Created triage coordinator agent, ID: {triage_agent.id}")
print("✓ Multi-agent system is ready!")

Created triage coordinator agent, ID: asst_ePAULa9RHpkzOwU5LxMk0mfU
✓ Multi-agent system is ready!


## Test the Multi-Agent System

Now let's test our multi-agent system with a support ticket. The triage agent will coordinate with the specialized agents to analyze the ticket.

In [8]:
# Create a thread for the conversation
print("Creating agent thread...")
thread = project_client.agents.threads.create()
print(f"Thread created, ID: {thread.id}")

# Create the ticket prompt
prompt = "Users can't reset their password from the mobile app."
print(f"\nSupport Ticket: {prompt}")

# Send a prompt to the agent
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"Message created, ID: {message.id}")

# Run the thread using the primary agent
print("\nProcessing ticket with multi-agent system. Please wait...")
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id, 
    agent_id=triage_agent.id
)

print(f"✓ Run finished with status: {run.status}")

if run.status == "failed":
    print(f"× Run failed: {run.last_error}")

Creating agent thread...
Thread created, ID: thread_5wvad7He0Qv27xqK3oRLhIMg

Support Ticket: Users can't reset their password from the mobile app.
Message created, ID: msg_iXMqd4JUdrFKDqjhMAKdOWAr

Processing ticket with multi-agent system. Please wait...
✓ Run finished with status: RunStatus.COMPLETED


## View the Triage Results

Let's retrieve and display the conversation to see how the agents collaborated to analyze the ticket.

In [9]:
# Fetch and display messages
messages = project_client.agents.messages.list(
    thread_id=thread.id, 
    order=ListSortOrder.ASCENDING
)

print("\n" + "="*60)
print("TICKET TRIAGE ANALYSIS")
print("="*60 + "\n")

for message in messages:
    if message.text_messages:
        last_msg = message.text_messages[-1]
        print(f"{message.role.upper()}:")
        print(f"{last_msg.text.value}\n")
        print("-"*60 + "\n")


TICKET TRIAGE ANALYSIS

USER:
Users can't reset their password from the mobile app.

------------------------------------------------------------

ASSISTANT:
The ticket regarding users being unable to reset their password from the mobile app has been assessed:

1. **Priority:** High. This is a user-facing issue affecting a critical functionality【message_idx:search_idx†source】.
2. **Assigned Team:** Frontend Team, as the issue pertains to the mobile app's interface【message_idx:search_idx†source】.
3. **Effort Required:** Medium. Implementing this feature will necessitate both backend and frontend updates to ensure secure password reset functionality, taking approximately 2-3 days to complete【message_idx:search_idx†source】.

------------------------------------------------------------



## Try Different Scenarios

You can test the multi-agent system with different support ticket scenarios. Here are some examples to try:

In [10]:
# Test different scenarios
test_scenarios = [
    "Investigate occasional 502 errors from the search endpoint.",
    "Update company logo on the homepage to match new branding.",
    "Database migration needed for new customer data schema.",
    "Fix typo in footer copyright year."
]

# Select a scenario to test (change the index to try different ones)
scenario_index = 0
test_prompt = test_scenarios[scenario_index]

print(f"Testing scenario: {test_prompt}\n")

# Create a new thread for this test
test_thread = project_client.agents.threads.create()

# Send the test prompt
test_message = project_client.agents.messages.create(
    thread_id=test_thread.id,
    role="user",
    content=test_prompt,
)

# Process the ticket
print("Processing ticket...")
test_run = project_client.agents.runs.create_and_process(
    thread_id=test_thread.id, 
    agent_id=triage_agent.id
)

# Display results
test_messages = project_client.agents.messages.list(
    thread_id=test_thread.id, 
    order=ListSortOrder.ASCENDING
)

print("\n" + "="*60)
print("SCENARIO TEST RESULTS")
print("="*60 + "\n")

for msg in test_messages:
    if msg.text_messages:
        last_msg = msg.text_messages[-1]
        print(f"{msg.role.upper()}:")
        print(f"{last_msg.text.value}\n")
        print("-"*60 + "\n")

Testing scenario: Investigate occasional 502 errors from the search endpoint.

Processing ticket...

SCENARIO TEST RESULTS

USER:
Investigate occasional 502 errors from the search endpoint.

------------------------------------------------------------

ASSISTANT:
The ticket regarding occasional 502 errors from the search endpoint is triaged as follows:

- **Priority:** High. This is a user-facing issue that affects functionality【message_idx:search_idx†source】.
- **Assigned Team:** Backend. The problem pertains to errors in the server-side functionality of the search endpoint【message_idx:search_idx†source】.
- **Effort Required:** Medium. The investigation will likely take 2-3 days to analyze logs, identify triggers, and potentially test under varying conditions. Coordination with infrastructure or backend teams might be necessary【message_idx:search_idx†source】.

------------------------------------------------------------



## Cleanup

When you're done testing, it's important to delete the agents to avoid unnecessary costs and resource usage.

In [11]:
# Clean up - Delete all agents
print("Cleaning up agents...")

try:
    project_client.agents.delete_agent(triage_agent.id)
    print("✓ Deleted triage agent")
    
    project_client.agents.delete_agent(priority_agent.id)
    print("✓ Deleted priority agent")
    
    project_client.agents.delete_agent(team_agent.id)
    print("✓ Deleted team agent")
    
    project_client.agents.delete_agent(effort_agent.id)
    print("✓ Deleted effort agent")
    
    print("\n✓ All agents successfully deleted!")
except Exception as e:
    print(f"× Error during cleanup: {str(e)}")

Cleaning up agents...
✓ Deleted triage agent
✓ Deleted priority agent
✓ Deleted team agent
✓ Deleted effort agent

✓ All agents successfully deleted!


## Summary

In this notebook, you've learned how to:

1. **Create specialized AI agents** - Each agent has a specific role (priority assessment, team assignment, effort estimation)
2. **Connect agents using tools** - Used `ConnectedAgentTool` to enable inter-agent communication
3. **Coordinate multiple agents** - Created a primary triage agent that orchestrates the specialized agents
4. **Process complex tasks** - Demonstrated how multiple agents can collaborate to analyze support tickets comprehensively

### Key Concepts

- **Multi-Agent Architecture**: Breaking down complex tasks into specialized roles
- **Connected Agents**: Agents can call other agents as tools to leverage their expertise
- **Orchestration**: A coordinator agent manages the workflow and combines insights from multiple agents
- **Separation of Concerns**: Each agent focuses on a single responsibility, making the system more maintainable and scalable

### Next Steps

You can extend this multi-agent system by:
- Adding more specialized agents (e.g., severity classifier, customer impact assessor)
- Implementing custom tools for specific business logic
- Integrating with real ticketing systems
- Adding monitoring and analytics capabilities

For more information, see the [Azure AI Foundry Agent Service documentation](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/agents).